# CDS Spread

## 1. Imports

In [2]:
import refinitiv.dataplatform.eikon as ek
ek.set_app_key('256eae2f881c499095090ac71dc96aa92cc6cb90')
import pandas as pd

ERROR!!! An error occurred while requesting URL('http://localhost:9000/api/status').
     ReadTimeout('')
ERROR!!! An error occurred while requesting URL('http://localhost:9000/api/status').
     ReadTimeout('')
ERROR!!! An error occurred while requesting URL('http://localhost:9060/api/status').
     ConnectError('All connection attempts failed')
Error: no proxy address identified.
Check if Desktop is running.
ERROR!!! An error occurred while requesting URL('http://localhost:9000/api/handshake').
     RemoteProtocolError("illegal status line: bytearray(b'\\xff\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x01\\x7fe\\x01')")


## 2. CDS Spread Extraction Function

In [2]:
def CDS_spread_table(companys, date, Tenors):
    frames=[]
    for baseCDSRIC, name in companys.items():
        CDSRICs = []

        for tenor in Tenors:
            CDSRICs.append(baseCDSRIC.replace("5Y", tenor))

        df,e = ek.get_data(CDSRICs,
                       ['TR.PARMIDSPREAD'],
                       {'SDate': date, 'EDate': date})
        df.drop(columns=['Instrument'], inplace=True)
        df.rename(columns={"Par Mid Spread": name}, inplace=True)
        frames.append(df)
        
    finalFrame = pd.concat(frames, axis=1)
    finalFrame.insert(loc=0, column='Tenor', value=Tenors)
    return finalFrame

## 3. Define Dictonary of Companys
Define a dictanory of the companys you are interested in. The keys are the RICs and the values are the names of the companies. The field "TR.CDSPrimaryCDSRic" returns the RIC for the 5Y CDS of the respective companies.

In [3]:
companys = {
    "DBKGn.DE": "Deutsche Bank",
    "RBIV.DE": "RBI",
    "CBKG.DE": "Commerzbank",
    "DGBGg.F^A13": "DZ Bank",
    "INGs.AS^I99": "ING",
    "CRIG.DE": "UniCredit",
    "HSBA.L": "HSBC",
    "BNPP.DE": "BNP",
    "JPM": "JPM",
    "BLGGgi.F^D20": "BayernLB",
    "LKBFg.F^I15": "LBBW",
    "GS": "GS",
    "LHTGg.F^E15": "Helaba"
}
df, err = ek.get_data(list(companys.keys()),'TR.CDSPrimaryCDSRic')
df.insert(loc=0, column='Name', value=list(companys.values()))
df

,Name,Instrument,Primary CDS RIC
0,Deutsche Bank,DBKGn.DE,DB5YEUAM=R
1,RBI,RBIV.DE,RIBH5YEUAM=R
2,Commerzbank,CBKG.DE,CBKG5YEUAM=R
3,DZ Bank,DGBGg.F^A13,<NA>
4,ING,INGs.AS^I99,INGB5YEUAM=R
5,UniCredit,CRIG.DE,UNIC5YEUAM=R
6,HSBC,HSBA.L,HSBA5YEUAM=R
7,BNP,BNPP.DE,BNPP5YEUAM=R
8,JPM,JPM,JPM5YUSAX=R
9,BayernLB,BLGGgi.F^D20,<NA>


## 4. Complete CDS RICs manually
For some companies the Eikon API will not return the 5Y CDS RIC. You have to look up these RICs in Refinitiv Workspace and add them manually. E.g., go to the company page on the tab "Debt & Credit"-> CDS Charts. In the upper right corner of the plot you should see the RIC of the 5Y CDS.

In [6]:
df.loc[3, "Primary CDS RIC"] = "DZ5YEUAM=R"
df.loc[9, "Primary CDS RIC"] = "BLB5YEUAM=R"
df.loc[10, "Primary CDS RIC"] = "LBBW5YEUAM=R"
df.loc[12, "Primary CDS RIC"] = "HELA5YEUAM=R"
df

,Name,Instrument,Primary CDS RIC
0,Deutsche Bank,DBKGn.DE,DB5YEUAM=R
1,RBI,RBIV.DE,RIBH5YEUAM=R
2,Commerzbank,CBKG.DE,CBKG5YEUAM=R
3,DZ Bank,DGBGg.F^A13,DZ5YEUAM=R
4,ING,INGs.AS^I99,INGB5YEUAM=R
5,UniCredit,CRIG.DE,UNIC5YEUAM=R
6,HSBC,HSBA.L,HSBA5YEUAM=R
7,BNP,BNPP.DE,BNPP5YEUAM=R
8,JPM,JPM,JPM5YUSAX=R
9,BayernLB,BLGGgi.F^D20,BLB5YEUAM=R


## 5. Call the CDS Spread Extraction Function

In [7]:
date = "2022-03-31"
Tenors = ['6M', '1Y', '2Y', '3Y', '4Y', '5Y', '7Y', '10Y']
RICdict = dict(zip(df.iloc[:, 2], df.iloc[:, 0]))
CDSframe = CDS_spread_table(RICdict, date, Tenors)
CDSframe

,Tenor,Deutsche Bank,RBI,Commerzbank,DZ Bank,ING,UniCredit,HSBC,BNP,JPM,BayernLB,LBBW,GS,Helaba
0,6M,38.56,174.43,29.08,42.83,10.61,43.55,23.09,17.6,32.9,24.09,34.48,37.63,34.88
1,1Y,38.58,176.36,29.09,47.72,10.62,43.57,23.1,17.61,37.1,30.04,39.4,41.81,37.83
2,2Y,47.26,183.9,38.05,54.59,16.85,64.69,34.84,25.34,42.65,36.59,44.38,53.47,43.75
3,3Y,55.83,192.78,46.9,60.34,23.02,74.72,46.44,32.98,49.78,46.36,49.36,63.03,49.66
4,4Y,66.89,198.52,57.5,63.82,29.21,88.65,58.53,41.14,58.51,56.08,54.34,74.84,55.56
5,5Y,78.87,207.03,68.53,65.58,36.4,103.43,70.53,49.77,67.42,67.87,59.3,85.21,61.44
6,7Y,92.19,221.25,78.5,68.04,43.63,118.96,86.36,58.4,79.39,80.1,67.87,101.95,70.42
7,10Y,104.34,232.59,85.92,70.39,49.85,136.04,100.96,66.97,95.14,100.43,76.36,114.56,79.13


## 6. Format the table into the bulk valuation input format and save as file

In [8]:
CDSframe.set_index('Tenor', inplace=True)
CDSframe = CDSframe.div(10000)
CDSframeT = CDSframe.T
CDSframeT.reset_index(inplace=True)
CDSframeT.rename(columns={"6M": "CDSSpread6M", 
                          "1Y": "CDSSpread1Y", 
                          "2Y": "CDSSpread2Y", 
                          "3Y": "CDSSpread3Y",
                          "4Y": "CDSSpread4Y",
                          "5Y": "CDSSpread5Y",
                          "7Y": "CDSSpread7Y",
                          "10Y": "CDSSpread10Y",
                          "index": "Customer"}, inplace=True)
CDSframeT.insert(loc=1, column='RecoveryRate', value=0.4)
CDSframeT["AsOfDate"] = "31.03.2022"
currency = ["EUR", "EUR", "EUR", "EUR", "EUR", "EUR", "EUR", "EUR", "USD", "EUR", "EUR", "USD", "EUR"]
CDSframeT["currency"] = currency
CDSframeT

Tenor,Customer,RecoveryRate,CDSSpread6M,CDSSpread1Y,CDSSpread2Y,CDSSpread3Y,CDSSpread4Y,CDSSpread5Y,CDSSpread7Y,CDSSpread10Y,AsOfDate,currency
0,Deutsche Bank,0.4,0.003856,0.003858,0.004726,0.005583,0.006689,0.007887,0.009219,0.010434,31.03.2022,EUR
1,RBI,0.4,0.017443,0.017636,0.01839,0.019278,0.019852,0.020703,0.022125,0.023259,31.03.2022,EUR
2,Commerzbank,0.4,0.002908,0.002909,0.003805,0.00469,0.00575,0.006853,0.00785,0.008592,31.03.2022,EUR
3,DZ Bank,0.4,0.004283,0.004772,0.005459,0.006034,0.006382,0.006558,0.006804,0.007039,31.03.2022,EUR
4,ING,0.4,0.001061,0.001062,0.001685,0.002302,0.002921,0.00364,0.004363,0.004985,31.03.2022,EUR
5,UniCredit,0.4,0.004355,0.004357,0.006469,0.007472,0.008865,0.010343,0.011896,0.013604,31.03.2022,EUR
6,HSBC,0.4,0.002309,0.00231,0.003484,0.004644,0.005853,0.007053,0.008636,0.010096,31.03.2022,EUR
7,BNP,0.4,0.00176,0.001761,0.002534,0.003298,0.004114,0.004977,0.00584,0.006697,31.03.2022,EUR
8,JPM,0.4,0.00329,0.00371,0.004265,0.004978,0.005851,0.006742,0.007939,0.009514,31.03.2022,USD
9,BayernLB,0.4,0.002409,0.003004,0.003659,0.004636,0.005608,0.006787,0.00801,0.010043,31.03.2022,EUR


In [34]:
# doesen't work properly
#CDSframeT.to_csv("Import_marketdata_CDS.csv", sep=";", decimal="," index=False)

In [61]:
CDSframeT.to_excel("Import_marketdata_CDS.xlsx", index=False)